In [18]:
from datetime import datetime
import chromedriver_autoinstaller

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import UnexpectedAlertPresentException
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import StaleElementReferenceException




import os
#os.system("pause")
import sys
import random
import csv
import time
import re
from bs4 import BeautifulSoup as bs


import pandas as pd

from datetime import datetime

In [19]:
#driver= webdriver.Chrome(r"C:\Users\danie\anaconda3\Lib\site-packages\chromedriver")
chromedriver_autoinstaller.install()
driver = webdriver.Chrome()

driver.get("https://www.google.com/")

In [20]:
url="https://www.linkedin.com/home"
driver.get(url)
time.sleep(3)

In [21]:
# Aceptar las cookies
#driver.find_element("xpath","/html/body/div/main/div[1]/div/section/div/div[2]/button[2]").click()
from selenium.webdriver.common.by import By
driver.find_element("xpath", "//*[@id='artdeco-global-alert-container']/div/section/div/div[2]/button[1]").click()
time.sleep(3)

In [22]:
#login
email=driver.find_element("xpath", "//input[@name='session_key']")
password=driver.find_element("xpath", "//input[@name='session_password']")

In [23]:
#leer email y password de un fichero externo
with open(r"email.txt") as myUser:
    username=myUser.read().replace('\n','')
email.send_keys(username)

with open(r"password.txt") as myPass:
    passcode=myPass.read().replace('\n','')
password.send_keys(passcode)
time.sleep(3)

In [24]:
submit=driver.find_element("xpath", "//button[@type='submit']").click()
time.sleep(2)

In [25]:
search_item='data analyst'

In [26]:
buscar=driver.find_element("xpath", "//input[@type='text']")
buscar.send_keys(search_item)
buscar.send_keys(Keys.RETURN)

In [27]:

#boton para acceder a la sección de empleos
time.sleep(6)
driver.find_element("xpath", "//*[@id='search-reusables__filters-bar']/ul/li[1]/button").click()
time.sleep(4)

'''
#boton empleos
#Utilizo el try porque cuesta encontrar el boton de empleos
try:
    driver.find_element("xpath", "//*[@id='search-reusables__filters-bar']/ul/li[1]/button").click()
    time.sleep(4)
except NoSuchElementException:
            time.sleep(4)
'''

'\n#boton empleos\n#Utilizo el try porque cuesta encontrar el boton de empleos\ntry:\n    driver.find_element("xpath", "//*[@id=\'search-reusables__filters-bar\']/ul/li[1]/button").click()\n    time.sleep(4)\nexcept NoSuchElementException:\n            time.sleep(4)\n'

In [28]:
# Recolectar los links (URLs) de cada una de las ofertas, en cada página (page) visitada
links = []
unique_hrefs = []
#El número de páginas a visitar se establece en el "page i range(x,x)"
print('Se están recolectando links en este momento...')
try: 
    for page in range(2,11):
        time.sleep(2)
        jobs_block = driver.find_elements(By.TAG_NAME,"jobs-search-results__list")
        jobs_list = driver.find_elements(By.CSS_SELECTOR,".jobs-search-results__list-item")

        url = driver.find_elements(By.TAG_NAME,'a')
        [unique_hrefs.append(x.get_attribute('href')) for x in url if x.get_attribute('href') not in unique_hrefs]

        for i in unique_hrefs:
            if i.startswith("https://www.linkedin.com/jobs/view"):
                links.append(i)
            else:
                pass
            # scroll down en cada ventana de oferta de trabajo
        for job in jobs_list:
            driver.execute_script("arguments[0].scrollIntoView();", job)

        print(f'Links recolectados en la página: {page-1} OK')
        # go to next page:
        driver.find_element("xpath",f"//button[@aria-label='Página {page}']").click()
        time.sleep(3)
except:
    pass
#print(links)
print(str(len(links)) + ' links encontrados en total')

Se están recolectando links en este momento...
Links recolectados en la página: 1 OK
Links recolectados en la página: 2 OK
Links recolectados en la página: 3 OK
Links recolectados en la página: 4 OK
Links recolectados en la página: 5 OK
Links recolectados en la página: 6 OK
Links recolectados en la página: 7 OK
Links recolectados en la página: 8 OK
Links recolectados en la página: 9 OK
360 links encontrados en total


In [29]:
#Antes de recolectar la información de cada uno de los links, se crean una serie de listas vacías para poder clasificar la información más adelante.

# listas vacias
job_titles = []
company_names = []
company_locations = []
work_methods = []
post_dates = []
work_times = [] 
job_desc = []

In [30]:
i = 0
j = 1

# Visistar cada uno de los links uno por uno para extraer el texto
print('En proceso...')
for i in range(len(links)):
    try:
        driver.get(links[i])
        i=i+1
        time.sleep(2)
        # Click en el boton ver más, para desplegar toda la información de la ventana.
        driver.find_element(By.CLASS_NAME,"artdeco-card__actions").click()
        time.sleep(2)
    except:
        pass
    
    # Selecciono tags específicos y presentes en todas las ofertas en Linkedin
    # Se utiliza la función try por si esos tags no etán presenten en alguna de las ofertas
    contents = driver.find_elements(By.CLASS_NAME,'p5')
    for content in contents:
        try:
            job_titles.append(driver.find_element(By.TAG_NAME,"h1").text)
            company_names.append(driver.find_element(By.CLASS_NAME,"jobs-unified-top-card__company-name").text)
            company_locations.append(driver.find_element(By.CLASS_NAME,"jobs-unified-top-card__bullet").text)
            work_methods.append(driver.find_element(By.CLASS_NAME,"jobs-unified-top-card__workplace-type").text)
            post_dates.append(driver.find_element(By.CLASS_NAME,"jobs-unified-top-card__posted-date").text)
            work_times.append(driver.find_element(By.CLASS_NAME,"jobs-unified-top-card__job-insight").text)
            print(f'Scraping the Job Offer {j} DONE.')
            j+= 1
            
        except:
            pass
        time.sleep(2)
        
        # Scraper del texto de la descripción del puesto
    job_description = driver.find_elements(By.CLASS_NAME,'jobs-description__content')
    for description in job_description:
        job_text = driver.find_element(By.CLASS_NAME,"jobs-box__html-content").text
        job_desc.append(job_text)
        print(f'Scraping the Job Offer {j}')
        time.sleep(2)  

En proceso...
Scraping the Job Offer 1 DONE.
Scraping the Job Offer 2
Scraping the Job Offer 2 DONE.
Scraping the Job Offer 3
Scraping the Job Offer 3 DONE.
Scraping the Job Offer 4
Scraping the Job Offer 4 DONE.
Scraping the Job Offer 5
Scraping the Job Offer 5 DONE.
Scraping the Job Offer 6
Scraping the Job Offer 6 DONE.
Scraping the Job Offer 7
Scraping the Job Offer 7 DONE.
Scraping the Job Offer 8
Scraping the Job Offer 8 DONE.
Scraping the Job Offer 9
Scraping the Job Offer 9 DONE.
Scraping the Job Offer 10
Scraping the Job Offer 10 DONE.
Scraping the Job Offer 11
Scraping the Job Offer 11 DONE.
Scraping the Job Offer 12
Scraping the Job Offer 12 DONE.
Scraping the Job Offer 13
Scraping the Job Offer 13 DONE.
Scraping the Job Offer 14
Scraping the Job Offer 14 DONE.
Scraping the Job Offer 15
Scraping the Job Offer 15 DONE.
Scraping the Job Offer 16
Scraping the Job Offer 16 DONE.
Scraping the Job Offer 17
Scraping the Job Offer 17 DONE.
Scraping the Job Offer 18
Scraping the Job 

In [31]:
# Creo el dataframe 
df = pd.DataFrame(list(zip(job_titles,company_names,
 company_locations,work_methods,
 post_dates,work_times,job_desc)),
 columns =['job_title', 'company_name',
 'company_location','work_method',
 'post_date','work_time','description'])

 #Para llevar un seguimiento en mi base de datos utilizo una columna con la fecha de hoy
df["fecha_scraping"]=pd.datetime.today().strftime("%d/%m/%Y")
#El id_job funcionará como id único para evitar duplicados cuando concatene scrapings de diferentes días en mi base de datos
df["id_job"]=df['job_title']+df['company_name']

C:\Users\danie\AppData\Local\Temp\ipykernel_5344\2848627743.py:10: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  df["fecha_scraping"]=pd.datetime.today().strftime("%d/%m/%Y")


In [32]:
df

,job_title,company_name,company_location,work_method,post_date,work_time,description,fecha_scraping,id_job
0,Becario Analista de datos,Xcalibur Multiphysics,"Madrid, Comunidad de Madrid, España",Presencial,hace 2 días,Jornada completa,Acerca del empleo\nGrupo Internacional líder e...,23/02/2023,Becario Analista de datosXcalibur Multiphysics
1,Data Analyst (Tres Cantos),Capitole Consulting,"Tres Cantos, Comunidad de Madrid, España",Híbrido,hace 1 semana,Jornada completa · Intermedio,Acerca del empleo\n¡Desde Capitole Consulting ...,23/02/2023,Data Analyst (Tres Cantos)Capitole Consulting
2,Analista de datos,CGI,Madrid y alrededores,Híbrido,hace 1 semana,Jornada completa · Intermedio,Acerca del empleo\n¡Únete a CGI!\n\n¿Quieres f...,23/02/2023,Analista de datosCGI
3,Data Analyst - Digital Customer,Bahia Principe Hotels & Resorts,"Palma, Islas Baleares, España",Híbrido,hace 1 semana,Jornada completa · Algo de responsabilidad,Acerca del empleo\nAre you passionate about da...,23/02/2023,Data Analyst - Digital CustomerBahia Principe ...
4,Data Analyst,Carrefour España,"Madrid, Comunidad de Madrid, España",Híbrido,hace 3 semanas,Jornada completa · Intermedio,Acerca del empleo\nDesde Carrefour tenemos la ...,23/02/2023,Data AnalystCarrefour España
...,...,...,...,...,...,...,...,...,...
334,Consultor Business Intelligence,Avanade,"Málaga, Andalucía, España",Híbrido,hace 6 días,Jornada completa · Intermedio,Acerca del empleo\nYOUR ROLE\n\nThe Data Opera...,23/02/2023,Consultor Business IntelligenceAvanade
335,Retail Business Analyst,Empresa Confidencial,Palma de Mallorca y alrededores,En remoto,hace 1 semana,Jornada completa · Sin experiencia,Acerca del empleo\nJob Description\n\nBuscamos...,23/02/2023,Retail Business AnalystEmpresa Confidencial
336,Business Intelligence Consultant,IQVIA Spain,"Madrid, Comunidad de Madrid, España",Presencial,hace 1 mes,Jornada completa · Sin experiencia,Acerca del empleo\nJob Details\n\nWhat You Wil...,23/02/2023,Business Intelligence ConsultantIQVIA Spain
337,Analytics Consultant,NATEEVO,"Madrid, Comunidad de Madrid, España",Híbrido,hace 18 horas,Jornada completa · Intermedio,Acerca del empleo\nJob Description\n\nBuscamos...,23/02/2023,Analytics ConsultantNATEEVO


In [33]:

# Guardo el dataframe en un csv
nombre_csv = 'ofertas_'+search_item + '_' + datetime.today().strftime('%Y-%m-%d') + ".csv"
df.to_csv(nombre_csv, index=False)

In [34]:
data = pd.read_csv(nombre_csv)
data

,job_title,company_name,company_location,work_method,post_date,work_time,description,fecha_scraping,id_job
0,Becario Analista de datos,Xcalibur Multiphysics,"Madrid, Comunidad de Madrid, España",Presencial,hace 2 días,Jornada completa,Acerca del empleo\nGrupo Internacional líder e...,23/02/2023,Becario Analista de datosXcalibur Multiphysics
1,Data Analyst (Tres Cantos),Capitole Consulting,"Tres Cantos, Comunidad de Madrid, España",Híbrido,hace 1 semana,Jornada completa · Intermedio,Acerca del empleo\n¡Desde Capitole Consulting ...,23/02/2023,Data Analyst (Tres Cantos)Capitole Consulting
2,Analista de datos,CGI,Madrid y alrededores,Híbrido,hace 1 semana,Jornada completa · Intermedio,Acerca del empleo\n¡Únete a CGI!\n\n¿Quieres f...,23/02/2023,Analista de datosCGI
3,Data Analyst - Digital Customer,Bahia Principe Hotels & Resorts,"Palma, Islas Baleares, España",Híbrido,hace 1 semana,Jornada completa · Algo de responsabilidad,Acerca del empleo\nAre you passionate about da...,23/02/2023,Data Analyst - Digital CustomerBahia Principe ...
4,Data Analyst,Carrefour España,"Madrid, Comunidad de Madrid, España",Híbrido,hace 3 semanas,Jornada completa · Intermedio,Acerca del empleo\nDesde Carrefour tenemos la ...,23/02/2023,Data AnalystCarrefour España
...,...,...,...,...,...,...,...,...,...
334,Consultor Business Intelligence,Avanade,"Málaga, Andalucía, España",Híbrido,hace 6 días,Jornada completa · Intermedio,Acerca del empleo\nYOUR ROLE\n\nThe Data Opera...,23/02/2023,Consultor Business IntelligenceAvanade
335,Retail Business Analyst,Empresa Confidencial,Palma de Mallorca y alrededores,En remoto,hace 1 semana,Jornada completa · Sin experiencia,Acerca del empleo\nJob Description\n\nBuscamos...,23/02/2023,Retail Business AnalystEmpresa Confidencial
336,Business Intelligence Consultant,IQVIA Spain,"Madrid, Comunidad de Madrid, España",Presencial,hace 1 mes,Jornada completa · Sin experiencia,Acerca del empleo\nJob Details\n\nWhat You Wil...,23/02/2023,Business Intelligence ConsultantIQVIA Spain
337,Analytics Consultant,NATEEVO,"Madrid, Comunidad de Madrid, España",Híbrido,hace 18 horas,Jornada completa · Intermedio,Acerca del empleo\nJob Description\n\nBuscamos...,23/02/2023,Analytics ConsultantNATEEVO
